### Name: Riad Mohammed
### Class: CSE 4510 Big Data
### Assignment: 3
### Due Date: Monday 20th October, 2022

#### Question 1) MapReduce on Movies (40 points)
#### Given a dataset named IMDB_Movies.csv, which is available on Canvas, use the Spark Resilient Distributed Dataset (RDD) model to count the number of times a country is involved in a movie. Your output will be key-pairs represented as [country, text-based bar-chart (count)] and sorted by bar-length. You may create the text-based bar-chart using an extended ASCII character such as the block character (ASCII character 219. See more ASCII characters here: https://theasciicode.com.ar/). For scaling, let 1 ASCII character = 1000 movies. Make sure to collect and display your results in your notebook using the rdd.collect() method.


In [1]:
#pip install pyspark

In [2]:
import findspark # makes pyspark work on my machine
findspark.init() # credit to Nikhil Agarwal on stackoverflow for this fix

In [3]:
%env PYSPARK_DRIVER_PYTHON=jupyter ./bin/pyspark

env: PYSPARK_DRIVER_PYTHON=jupyter ./bin/pyspark


In [4]:
from pyspark import SparkContext, SparkConf
conf = SparkConf() \
     .setAppName('app') \
     .setMaster('local[*]')
sc = SparkContext(conf=conf)

In [5]:
#pip install findspark

In [6]:
import pandas as pd
#reads csv data from excel
movies = pd.read_csv("IMDb_movies.csv")

C:\Users\Riad\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
movies.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [8]:
#Get the country columns with the respestive countries
country = movies["country"]

In [9]:
country

0                     USA
1               Australia
2        Germany, Denmark
3                     USA
4                   Italy
               ...       
85850     France, Belgium
85851         Netherlands
85852               India
85853              Turkey
85854               Spain
Name: country, Length: 85855, dtype: object

In [10]:
#converts column to a list to be manipulated
new_country = country.to_list()

In [11]:
new_country

['USA',
 'Australia',
 'Germany, Denmark',
 'USA',
 'Italy',
 'USA',
 'Germany',
 'Italy',
 'Romania',
 'France, USA',
 'Denmark',
 'France',
 'Sweden',
 'France',
 'Italy',
 'Belgium',
 'France',
 'USA',
 'Germany',
 'USA',
 'Italy',
 'Italy',
 'USA',
 'USA',
 'Italy',
 'USA',
 'France',
 'France',
 'USA',
 'Germany',
 'Denmark',
 'USA',
 'USA',
 'USA, Canada',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'Hungary',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'Denmark',
 'USA',
 'Italy',
 'USA',
 'USA',
 'Russia',
 'Italy',
 'USA',
 'France',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'Sweden',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'France',
 'USA',
 'Russia',
 'USA',
 'USA',
 'Germany',
 'USA',
 'USA',
 'USA',
 'USA',
 'Sweden',
 'USA',
 'USA',
 'USA',
 'USA',
 'Germany',
 'USA',
 'Denmark',
 'USA',
 'USA',
 'USA',
 'France',
 'USA',
 'Russia',
 'USA',
 'USA',
 'USA',
 'USA',
 'Russia',
 'USA',


In [12]:
#Removes whitespaces present within the list to ensure all countries are read
new_list = [str(item).replace(' ', '') for item in new_country]

In [13]:
new_list

['USA',
 'Australia',
 'Germany,Denmark',
 'USA',
 'Italy',
 'USA',
 'Germany',
 'Italy',
 'Romania',
 'France,USA',
 'Denmark',
 'France',
 'Sweden',
 'France',
 'Italy',
 'Belgium',
 'France',
 'USA',
 'Germany',
 'USA',
 'Italy',
 'Italy',
 'USA',
 'USA',
 'Italy',
 'USA',
 'France',
 'France',
 'USA',
 'Germany',
 'Denmark',
 'USA',
 'USA',
 'USA,Canada',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'Hungary',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'Denmark',
 'USA',
 'Italy',
 'USA',
 'USA',
 'Russia',
 'Italy',
 'USA',
 'France',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'Sweden',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'France',
 'USA',
 'Russia',
 'USA',
 'USA',
 'Germany',
 'USA',
 'USA',
 'USA',
 'USA',
 'Sweden',
 'USA',
 'USA',
 'USA',
 'USA',
 'Germany',
 'USA',
 'Denmark',
 'USA',
 'USA',
 'USA',
 'France',
 'USA',
 'Russia',
 'USA',
 'USA',
 'USA',
 'USA',
 'Russia',
 'USA',
 'U

In [14]:
with open(r'C:/Users/Riad/Desktop/cse4510_Big_Data_assignments/assignment_3/countries.txt','w') as fp:
    for item in new_list:
        fp.write("%s\n" %item)
    print('Done')

Done


In [15]:
country_file = '/Users/Riad/Desktop/cse4510_Big_Data_assignments/assignment_3/countries.txt'

In [16]:
#read data from text file and split each line into words
words = sc.textFile(country_file).flatMap(lambda line: line.split(","))

#count the occurrence of each word
wordCounts = words.map(lambda word: (word, 1)).reduceByKey(lambda a, b:a + b)

In [17]:
words.collect()

['USA',
 'Australia',
 'Germany',
 'Denmark',
 'USA',
 'Italy',
 'USA',
 'Germany',
 'Italy',
 'Romania',
 'France',
 'USA',
 'Denmark',
 'France',
 'Sweden',
 'France',
 'Italy',
 'Belgium',
 'France',
 'USA',
 'Germany',
 'USA',
 'Italy',
 'Italy',
 'USA',
 'USA',
 'Italy',
 'USA',
 'France',
 'France',
 'USA',
 'Germany',
 'Denmark',
 'USA',
 'USA',
 'USA',
 'Canada',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'Hungary',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'Denmark',
 'USA',
 'Italy',
 'USA',
 'USA',
 'Russia',
 'Italy',
 'USA',
 'France',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'Sweden',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'France',
 'USA',
 'Russia',
 'USA',
 'USA',
 'Germany',
 'USA',
 'USA',
 'USA',
 'USA',
 'Sweden',
 'USA',
 'USA',
 'USA',
 'USA',
 'Germany',
 'USA',
 'Denmark',
 'USA',
 'USA',
 'USA',
 'France',
 'USA',
 'Russia',
 'USA',
 'USA',
 'USA',
 'USA',
 'Russia',

In [18]:
wordCounts.collect()

[('USA', 34325),
 ('Australia', 1181),
 ('Denmark', 1026),
 ('France', 8311),
 ('Belgium', 1354),
 ('Hungary', 642),
 ('Mexico', 1173),
 ('Norway', 639),
 ('Austria', 448),
 ('India', 6373),
 ('Switzerland', 636),
 ('China', 1166),
 ('Brazil', 979),
 ('Portugal', 429),
 ('Argentina', 853),
 ('Poland', 902),
 ('Netherlands', 1031),
 ('Finland', 676),
 ('WestGermany', 1114),
 ('Greece', 769),
 ('EastGermany', 74),
 ('Albania', 40),
 ('Monaco', 26),
 ('Ireland', 476),
 ('Philippines', 384),
 ('PuertoRico', 38),
 ('Algeria', 47),
 ('Lebanon', 65),
 ('Bermuda', 2),
 ('Ukraine', 141),
 ('Mauritania', 5),
 ('Iran', 744),
 ('Thailand', 361),
 ('Iceland', 163),
 ('Venezuela', 64),
 ('Peru', 76),
 ('Tunisia', 54),
 ('Angola', 8),
 ('Ecuador', 17),
 ('Andorra', 3),
 ('Ethiopia', 9),
 ('Libya', 2),
 ('Kuwait', 5),
 ('Suriname', 1),
 ('Vietnam', 64),
 ('Cyprus', 30),
 ('Botswana', 2),
 ('Nigeria', 35),
 ('Kenya', 17),
 ('CostaRica', 23),
 ('Zambia', 3),
 ('BurkinaFaso', 15),
 ('Iraq', 24),
 ('Aruba

In [19]:
#temp variable created from key pairs for further manipulation
new_word = wordCounts.collect()

In [20]:
#function to calculate the bar length based country count
def calBar(x):
    char ='█'
    bars = x // 1000
    real_bar=(bars*char)
    return real_bar


In [21]:
#function to calculate counts in 1k abbreviation
def formatNum(x):
    x = float(x / 1000)
    temp = str(round(x, 2))
    s = temp + 'k'
    return s

In [22]:
formatNum(34325)

'34.33k'

In [23]:
#Displays the output before counts are sorted in descending order
output = wordCounts.map(lambda x: (x[0], calBar(x[1]),formatNum(x[1])))

In [24]:
output.collect()

[('USA', '██████████████████████████████████', '34.33k'),
 ('Australia', '█', '1.18k'),
 ('Denmark', '█', '1.03k'),
 ('France', '████████', '8.31k'),
 ('Belgium', '█', '1.35k'),
 ('Hungary', '', '0.64k'),
 ('Mexico', '█', '1.17k'),
 ('Norway', '', '0.64k'),
 ('Austria', '', '0.45k'),
 ('India', '██████', '6.37k'),
 ('Switzerland', '', '0.64k'),
 ('China', '█', '1.17k'),
 ('Brazil', '', '0.98k'),
 ('Portugal', '', '0.43k'),
 ('Argentina', '', '0.85k'),
 ('Poland', '', '0.9k'),
 ('Netherlands', '█', '1.03k'),
 ('Finland', '', '0.68k'),
 ('WestGermany', '█', '1.11k'),
 ('Greece', '', '0.77k'),
 ('EastGermany', '', '0.07k'),
 ('Albania', '', '0.04k'),
 ('Monaco', '', '0.03k'),
 ('Ireland', '', '0.48k'),
 ('Philippines', '', '0.38k'),
 ('PuertoRico', '', '0.04k'),
 ('Algeria', '', '0.05k'),
 ('Lebanon', '', '0.07k'),
 ('Bermuda', '', '0.0k'),
 ('Ukraine', '', '0.14k'),
 ('Mauritania', '', '0.01k'),
 ('Iran', '', '0.74k'),
 ('Thailand', '', '0.36k'),
 ('Iceland', '', '0.16k'),
 ('Venezuela',

In [25]:
#Displays the output after counts are sorted in descending order
finalOutput = output.sortBy(lambda x:len(x[1]),ascending = False)

In [26]:
#Output to match the desired output from the assignment page
finalOutput.take(19)

[('USA', '██████████████████████████████████', '34.33k'),
 ('France', '████████', '8.31k'),
 ('UK', '███████', '7.49k'),
 ('India', '██████', '6.37k'),
 ('Italy', '█████', '5.06k'),
 ('Germany', '███', '3.72k'),
 ('Canada', '███', '3.62k'),
 ('Japan', '███', '3.7k'),
 ('Spain', '██', '2.73k'),
 ('Australia', '█', '1.18k'),
 ('Denmark', '█', '1.03k'),
 ('Belgium', '█', '1.35k'),
 ('Mexico', '█', '1.17k'),
 ('China', '█', '1.17k'),
 ('Netherlands', '█', '1.03k'),
 ('WestGermany', '█', '1.11k'),
 ('Sweden', '█', '1.23k'),
 ('Russia', '█', '1.08k'),
 ('HongKong', '█', '1.88k')]

In [ ]:
# save the counts to output file
finalOutput.saveAsTextFile("data/count")